In [3]:
import pandas as pd
#import yfinance as yf
from yahoofinancials import YahooFinancials as yfs
import json

# !pip install yahoofinancials

In [4]:
def Sheet_Data(ticker,sheet_data):
    save = []
    
    df = pd.DataFrame.from_dict({(i,j): sheet_data[i][j] 
                            for i in sheet_data.keys() 
                            for j in sheet_data[i].keys()},
                        orient='index')

    for i in df.columns:
        f = len(df.columns) - i - 1
        df2 = pd.json_normalize(df[f],max_level = 1)

        #Gets  Date and renames 
        Date = df2.columns[0]
        Date = Date[0:11]
        df2.index = df2.index.rename(ticker)
        df2.columns = df2.columns.str.replace(str(Date),'')

        #Tranposes DataFrame and changes it to the 
        df2 = df2.T
        df2.rename(columns = {df2.columns[0]:'{}'.format(str(Date[0:4]))},inplace = True)
        #display(df2)
        save.append(df2)
    df3 = pd.concat(save,axis = 1)
    #display(df3)
    filename = str(df.index[0][1]) + '_' + str(df.index[0][0])
    print(filename)
    #df3.to_csv('{}.csv'.format(filename))
    return df3
#filename = str(df.index[0][1]) + '_' + str(df.index[0][0])
#df3.t

In [5]:
ticker = 'AAPL'
yahoo_financials = yfs(ticker)

cash = yahoo_financials.get_financial_stmts('annual','cash')
income = yahoo_financials.get_financial_stmts('annual','income')
balance = yahoo_financials.get_financial_stmts('annual','balance')
stock = yahoo_financials.get_key_statistics_data()[ticker]

cash_balance = Sheet_Data(ticker,cash)
income_balance = Sheet_Data(ticker,income)
balance_balance = Sheet_Data(ticker,balance)

AAPL_cashflowStatementHistory
AAPL_incomeStatementHistory
AAPL_balanceSheetHistory


In [6]:
#WACC = Wd * Rd * (1 - t) + We + Re
INCOME2 = income_balance.loc[['incomeBeforeTax','incomeTaxExpense']]
t = income_balance.loc['incomeBeforeTax']/ income_balance.loc['incomeTaxExpense']
t = t[-1]/ 100 



yahoo_financials.get_beta()
df = pd.read_html('https://finance.yahoo.com/bonds')
Rf = float(df[0]['Last Price'][2])
try: 
    beta = float(yahoo_financials.get_beta())
except:
    beta = 0
Rm = float(10.0)
Ra = Rf + beta * (Rm - Rf)
#print(Ra)

try:
    Balance2 = balance_balance.loc[['shortLongTermDebt','longTermDebt']]#.astype#(float)
except:
    Balance2 = balance_balance.loc[['longTermDebt']]

Total_debt = Balance2.sum()[-1] 
Balance2 =   -(income_balance.loc['interestExpense'][-1]) / Total_debt
Rd = (1-Balance2) * t #  Rd * (1 - t)
#Balance2.sum()


Market_Cap = yahoo_financials .get_market_cap() # MarketCap
Total =  Total_debt + Market_Cap 
Wd = Total_debt/ Total 
We = Market_Cap/ Total 
Re = Ra
Required_Rate = (Wd * Rd + We * Re)/100
Required_Rate

FCF = cash_balance.loc[['totalCashFromOperatingActivities','capitalExpenditures','netIncome']] # how to select rows 
Revenue = income_balance.loc[['totalRevenue','netIncome']]
Growth_Rate = Revenue.pct_change().mean()
#FCF is CFO - CAPEX = FCF
#FCF = cash_balance.loc[['totalCashFromOperatingActivities','capitalExpenditures','netBorrowings']] # More 

df = pd.DataFrame()
df['FCF'] = FCF.loc['totalCashFromOperatingActivities'] + FCF.loc['capitalExpenditures']
df['Net Income'] = FCF.loc['netIncome']
#df['FCF/Net Income'] = df['FCF']/df['Net Income']
minimum = (df['FCF']/df['Net Income']).min()
print(minimum)
df

# Anaylsis

In [12]:
def DCF_VALUE():
    outstanding_shares = stock['sharesOutstanding']

    Required_rate = Required_Rate
    Pre_rate = .025
    Cashflowgrowthrate = minimum

    years = [1,2,3,4]
    freecash_flow = df['FCF'].values

    terminalvalue = freecash_flow[-1] * (1+Pre_rate)/(Required_rate - Pre_rate)

    futurefreecashflow = []
    discountfactor = []
    discountfuturefreecashflow = []
    for  year in years:
        cashflow = freecash_flow[-1] * (1+ Cashflowgrowthrate) ** year 
        futurefreecashflow.append(cashflow)
        discountfactor.append((1+ Required_rate)**year)


    for i in range(0,len(years)):
        discountfuturefreecashflow.append(futurefreecashflow[i]/discountfactor[i])

    print(discountfuturefreecashflow)
    dis = terminalvalue/(1+Required_rate)**len(years)
    discountfuturefreecashflow.append(dis)
    value = sum(discountfuturefreecashflow)
    fairvalue = value/outstanding_shares

    print(ticker)
    print(fairvalue)
    print(yahoo_financials.get_current_price())

    print(Cashflowgrowthrate)
    print(Required_Rate)
#print(stock.get_current_price())
DCF_VALUE()

[135933139095.8545, 251861491234.96118, 466657440481.5829, 864638598338.4235]
AAPL
134.60911316919862
121.03
1.065875199073404
0.11498149007759104


In [10]:
def DCF_VALUE_CON():
    outstanding_shares = stock['sharesOutstanding']

    Required_rate = .075
    Pre_rate = .025
    Cashflowgrowthrate = .035

    years = [1,2,3,4]
    freecash_flow = df['FCF'].values

    terminalvalue = freecash_flow[-1] * (1+Pre_rate)/(Required_rate - Pre_rate)

    futurefreecashflow = []
    discountfactor = []
    discountfuturefreecashflow = []
    for  year in years:
        cashflow = freecash_flow[-1] * (1+ Cashflowgrowthrate) ** year 
        futurefreecashflow.append(cashflow)
        discountfactor.append((1+ Required_rate)**year)


    for i in range(0,len(years)):
        discountfuturefreecashflow.append(futurefreecashflow[i]/discountfactor[i])

    print(discountfuturefreecashflow)
    dis = terminalvalue/(1+Required_rate)**len(years)
    discountfuturefreecashflow.append(dis)
    value = sum(discountfuturefreecashflow)
    fairvalue = value/outstanding_shares

    print(ticker)
    print(fairvalue)
    print(yahoo_financials.get_current_price())

    print(Cashflowgrowthrate)
    print(Required_rate)
#print(stock.get_current_price())
DCF_VALUE_CON()

[70635139534.88373, 68006855273.1206, 65476367635.050995, 63040037676.53747]
AAPL
82.99576991597596
121.03
0.035
0.075


In [13]:
df = pd.read_html('https://www.marketwatch.com/investing/stock/sq/financials/balance-sheet/quarter')
df[4]

,Item Item,31-Dec-2019,31-Mar-2020,30-Jun-2020,30-Sep-2020,31-Dec-2020,5- qtr trend
0,Cash & Short Term Investments Cash & Short Te...,1.58B,2.52B,2.71B,2.91B,3.88B,NaN
1,Cash & Short Term Investments Growth Cash & S...,-,59.55%,7.48%,7.46%,33.51%,NaN
2,Cash Only Cash Only,1.09B,2B,1.99B,2.15B,3.19B,NaN
3,Short-Term Investments Short-Term Investments,-,-,-,-,-,NaN
4,Cash & ST Investments / Total Assets Cash & S...,34.68%,41.95%,34.65%,35.85%,39.35%,NaN
5,Total Accounts Receivable Total Accounts Rece...,689.84M,637.19M,1.03B,1.03B,1.22B,NaN
6,Total Accounts Receivable Growth Total Accoun...,-,-7.63%,62.01%,0.11%,17.62%,NaN
7,"Accounts Receivables, Net Accounts Receivable...",622.56M,553.14M,917.63M,886.4M,1.07B,NaN
8,"Accounts Receivables, Gross Accounts Receivab...",622.56M,553.14M,917.63M,886.4M,1.07B,NaN
9,Bad Debt/Doubtful Accounts Bad Debt/Doubtful ...,-,-,-,-,-,NaN


In [246]:
import pandas as pd
import requests
def Balance_SHEET_ZACK():
    i = 'AAPL'
    url = ("https://www.zacks.com/stock/quote/{}/balance-sheet".format(i))
    headers = {'User-Agent': 'Mozilla/5.0'}
    result = requests.get(url, headers=headers)
    df2 = pd.read_html(result.text)
    return df2

In [247]:
def Income_SHEET_ZACK():
    i = 'AAPL'
    url = ("https://www.zacks.com/stock/quote/{}/income-statement".format(i))
    headers = {'User-Agent': 'Mozilla/5.0'}
    result = requests.get(url, headers=headers)
    df2 = pd.read_html(result.text)
    return df2

In [242]:
df2 = Balance_SHEET_ZACK()

In [190]:
#display(df2[1])
#display(df2[2])
#display(df2[3])
#display(df2[4])
#display(df2[5])
#display(df2[6])
#display(df2[5])
#display(df2[6])
#display(df2[7])
x = []
x.append(df2[2].columns[0])
x.append(df2[3].columns[0])
x.append(df2[4].columns[0])
x

['Unnamed: 0', 'Depreciation Footnote', 'Earnings Per Share Data']

In [248]:
def Sheets(df2,name):
    df = pd.concat([df2[2],df2[3],df2[4]])
    x = []
    x.append(df2[2].columns[0])
    x.append(df2[3].columns[0])
    x.append(df2[4].columns[0])

    y = []
    y.append(df[str(x[0])].dropna())
    y.append(df[str(x[1])].dropna())
    y.append(df[str(x[2])].dropna())
    y = pd.concat(y).tolist()


    df = pd.concat([df2[2],df2[3],df2[4]])
    df['{}'.format(name)] = y
    df = df.drop(columns= x)
    df = df.set_index('{}'.format(name))
    display(df)
Sheets(Balance_SHEET_ZACK(),'Balance Sheet')
Sheets(Income_SHEET_ZACK(),'Income Sheet')

,9/30/2020,9/30/2019,9/30/2018,9/30/2017,9/30/2016
Balance Sheet,,,,,
Assets,NaN,NaN,NaN,NaN,NaN
Cash & Equivalents,41601.00,41601.00,66301.00,74181.00,67155.00
Receivables,30343.00,30343.00,48995.00,35673.00,29299.00
Notes Receivable,0.00,0.00,0.00,0.00,0.00
Inventories,2349.00,2349.00,3956.00,4855.00,2132.00
Other Current Assets,15085.00,15085.00,12087.00,13936.00,8283.00
Total Current Assets,89378.00,89378.00,131339.00,128645.00,106869.00
Net Property & Equipment,22471.00,22471.00,41304.00,33783.00,27010.00
Investments & Advances,164065.00,164065.00,170799.00,194714.00,170430.00


,9/30/20,9/30/19,9/30/18,9/30/17,9/30/16
Income Sheet,,,,,
Sales,233715.00,233715.00,265595.00,229234.00,215639.00
Cost Of Goods,140089.00,140089.00,163756.00,141048.00,131376.00
Gross Profit,93626.00,93626.00,101839.00,88186.00,84263.00
Selling & Adminstrative & Depr. & Amort Expenses,22396.00,22396.00,30941.00,26842.00,24239.00
Income After Depreciation & Amortization,71230.00,71230.00,70898.00,61344.00,60024.00
Non-Operating Income,1285.00,1285.00,2005.00,2745.00,1348.00
Interest Expense,0.00,0.00,0.00,0.00,0.00
Pretax Income,72515.00,72515.00,72903.00,64089.00,61372.00
Income Taxes,19121.00,19121.00,13372.00,15738.00,15685.00


In [ ]:
'Interest Expense','Long-Term Debt','Pretax Income','Income Taxes', 'Sales'
#Sales = Revenue

In [240]:
df = pd.concat([df2[2],df2[3],df2[4]])
x = []
x.append(df2[2].columns[0])
x.append(df2[3].columns[0])
x.append(df2[4].columns[0])

y = []
y.append(df[str(x[0])].dropna())
y.append(df[str(x[1])].dropna())
y.append(df[str(x[2])].dropna())
y = pd.concat(y).tolist()


df = pd.concat([df2[2],df2[3],df2[4]])
df['Income Sheet'] = y
df = df.drop(columns= x)
df = df.set_index('Income Sheet')
df

,12/31/20,12/31/19,12/31/18,12/31/17,12/31/16
Income Sheet,,,,,
Sales,1267.00,1267.00,3298.00,2214.00,1709.00
Cost Of Goods,897.00,897.00,1994.00,1375.00,1133.00
Gross Profit,370.00,370.00,1304.00,839.00,576.00
Selling & Adminstrative & Depr. & Amort Expenses,544.00,544.00,1340.00,894.00,746.00
Income After Depreciation & Amortization,-174.00,-174.00,-37.00,-54.00,-170.00
Non-Operating Income,0.00,0.00,18.00,0.00,0.00
Interest Expense,1.00,1.00,18.00,8.00,-1.00
Pretax Income,-176.00,-176.00,-36.00,-63.00,-170.00
Income Taxes,4.00,4.00,2.00,0.00,2.00


df = pd.concat([df2[2],df2[3],df2[4]])
#display(df)
x = []
x.append(df['Unnamed: 0'].dropna())
x.append(df['Liabilities & Shareholders Equity'].dropna())
x.append(df['Shareholders Equity'].dropna())

y = pd.concat(x).tolist()
df = df.drop(['Liabilities & Shareholders Equity','Shareholders Equity','Unnamed: 0'], axis=1)
df['Liabilities & Shareholders Equity'] = y
df.set_index('Liabilities & Shareholders Equity')

x#Anaylist esitmates 
df = pd.read_html('https://finance.yahoo.com/quote/AAPL/analysis?p=AAPL')
df = df[1] 
df.index = df[df.columns[0]]
df1 = df.index 
df1 = df[df.columns[3:5]]
df1 = df1.drop(['Sales Growth (year/est)'])

for i in df1.columns:
    df1[i] = df1[i].str.replace('B','e9')
    df1[i] = df1[i].astype(float)

#df1.loc['Avg. Estimate']
df1.loc['Avg. Estimate'].tolist()

Revenue1 = Revenue.loc['totalRevenue']
strs = int(Revenue.columns[3])
Revenue1['{}'.format(strs+1)] = df1.loc['Avg. Estimate'][0]
Revenue1['{}'.format(strs+2)] = df1.loc['Avg. Estimate'][1]
growth = 1 + Revenue.T.pct_change().sum()/5

Revenue1['{}'.format(strs+3)] = l.drop_index() * growth
Revenue1['{}'.format(strs+4)] = l2.drop_index() * growth
Revenue.loc['netIncome']/Revenue.loc['totalRevenue']

ticker = 'LMT'
yahoo_financials = yfs(ticker)

cash = yahoo_financials.get_financial_stmts('annual','cash')
income = yahoo_financials.get_financial_stmts('annual','income')
balance = yahoo_financials.get_financial_stmts('annual','balance')
stock = yahoo_financials.get_key_statistics_data()[ticker]

sample = pd.DataFrame()
sample['Free Cash Flow'] = df['FCF']
sample['Cash Flow'] = futurefreecashflow
sample['discountfactor'] = discountfactor
sample

df = pd.DataFrame.from_dict({(i): stock[i] 
                            for i in stock.keys() 
                            },
                        orient='index')
df

aapl.info['sharesOutstanding']
stock.keys()

pd.DataFrame.from_dict({(i): stock[i] 
for i in stock.keys()},orient='index')

# Testing for working all of the files work

df = pd.DataFrame.from_dict({(i,j): balance_sheet_data[i][j] 
                           for i in balance_sheet_data.keys() 
                           for j in balance_sheet_data[i].keys()},
                       orient='index')
for i in df.columns:
    df[i].tolist()
    df2 = pd.json_normalize(df[i],max_level = 1)

    #Gets  Date and renames 
    Date = df2.columns[0]
    Date = Date[0:11]
    df2.index = df2.index.rename(ticker)
    df2.columns = df2.columns.str.replace(str(Date),'')

    #Tranposes DataFrame and changes it to the 
    df2 = df2.T
    df2.rename(columns = {df2.columns[0]:'{}'.format(str(Date[0:10]))},inplace = True)
    #display(df2)
    save.append(df2)
df3 = pd.concat(save,axis = 1)
display(df3)